In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 인공지능 각본가/작사가 만들기

# 0. 인공지능의 작문

#### 현존하는 최고의 인공지능 작문가 GPT-2(https://talktotransformer.com/)  
- GPT-2 이전에도 작문을 할 수 있는 딥러닝 모델은 존재했지만, 생성한 문장 길이가 일정 길이 이상이 되면, 주제의 일관성이 흐트러지고 어색해짐
- 하지만 GPT-2는 신문기사 1편 정도의 길이의 글을 작문하면서, **주제나 논리의 일관성을 어느정도 유지**
- 19년 2월, GPT-2를 발표한 OpenAI에서 **문장 생성 모델의 오남용이 가져올 위험 때문에 해당 모델을 비공개**하기로 하면서 [기사](https://decenter.kr/NewsView/1VFGQMBBXZ/GZ02)들이 한동안 세간의 이슈가 되었음
- 그로부터 1년여 후 2020년 5월에 OpenAI에서는 다시 GPT-2를 이전보다 훨씬 큰 규모로 발전시킨 **GPT-3**를 발표. GPT-3가 만들어낸 텍스트는 그저 논리적 일관성을 유지하는 수준을 넘어서서 **사람이 쓴 것과 구분이 안될 정도의 자연스러움**을 보여줌

#### 그렇다면 왜 작문하는 인공지능이 이토록 충격을 주는 것일까?
- 인공지능이 사람의 언어에서 생각과 의도와 감정을 읽어 내고, 그 의미를 이해하며, 적절한 말을 만들어내서 인간의 질문에 대답할 수 있다면 그야말로 지능을 가진 기계라는 특이점에 이르렀다고 할 수 있기 때문




#### 🔥 **인공지능이 문장을 이해하는 방식과 작문을 가르치는 법을 공부하고, 구현해 보자 !** 🔥


# 1. 인공지능 각본가 만들기

## 1-1. 데이터 전처리
① 정규표현식을 이용한 corpus 생성  
② `tf.keras.preprocessing.text.Tokenizer`를 이용해 corpus를 텐서로 변환  
③ `tf.data.Dataset.from_tensor_slices()`를 이용해 corpus 텐서를 `tf.data Dataset`객체로 변환

### 1-1-1. 데이터 확인

In [3]:
import re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = '/content/drive/MyDrive/Colab Notebooks/Data-Science/data/lyricist/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


문장(대사) 만을 원하므로 화자 이름이나 공백뿐인 정보는 필요없음
- 화자가 표기된 문장 (`~:`)
- 공백인 문장(`len=0`) 제거

In [4]:
#cnt=0
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.
    
    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
    #cnt+=1    
    print(sentence)
#print(cnt) # 총 문장수 : 24015

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


### 1-2-1. Tokenize

텍스트 생성 모델의 경우에도 단어 사전을 만듦 => **문장을 일정한 기준으로 쪼개기**  

**공백을 기준으로 토큰화하자!**  
주의사항
1. Hi, my name is John. *("Hi," "my", ..., "john." 으로 분리됨) - 문장부호  
  -> 문장 부호 양쪽에 공백 추가

2. First, open the first chapter. *(First와 first를 다른 단어로 인식) - 대소문자  
  -> 모든 문자들을 소문자로 변환
3. He is a ten-year-old boy. *(ten-year-old를 한 단어로 인식) - 특수문자  
  -> 특수문자 모두 제거

In [5]:
# 입력된 문장을
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


**소스 문장(Source Sentence)** : 모델의 입력이 되는 문장. `X_train`  
**타겟 문장(Target Sentence)** : 정답 역할을 하게 될 모델의 출력 문장. `y_train`

### 1-2-2. 정제 데이터 구축

정제 함수를 통해 만든 데이터셋에서 토큰화를 진행한 후 끝 단어 <end>를 없애면 **소스 문장**, 첫 단어 <start>를 없애면 타겟 문장

In [6]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> before we proceed any further , hear me speak . <end>',
 '<start> speak , speak . <end>',
 '<start> you are all resolved rather to die than to famish ? <end>',
 '<start> resolved . resolved . <end>',
 '<start> first , you know caius marcius is chief enemy to the people . <end>',
 '<start> we know t , we know t . <end>',
 '<start> let us kill him , and we ll have corn at our own price . <end>',
 '<start> is t a verdict ? <end>',
 '<start> no more talking on t let it be done away , away ! <end>',
 '<start> one word , good citizens . <end>']

### 1-3-1. Vectorize

**데이터 사전을 활용해 언어 데이터를 숫자로 변환**  
인공지능 (숫자) <-> 사람 (언어)  


`tf.keras.preprocessing.text.Tokenizer`를 활용해 vectorize 하자
- 정제된 데이터를 토큰화
- 단어 사전(vocabulary 또는 dictionary라고 칭함) 만듦
- 데이터를 숫자로 변환 (변환된 데이터 : `tensor`)
  - [Tensor란 무엇인가?](https://rekt77.tistory.com/102)

In [7]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f411b435410>


텐서 데이터 : tokenizer에 구축된 단어 사전의 인덱스 (type int)


In [8]:
# 단어 사전을 확인해보자
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break
print(143,":", tokenizer.index_word[143])    

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of
143 : before


### 1-3-2. vectorize 된 tensor를 소스와 타겟으로 분리

In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0]) # <start>~~ .
print(tgt_input[0]) # ~~~ <end>(또는 <pad>)

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


텐서 출력부에서 행 뒤쪽에 0이 많이 나온 부분은 정해진 입력 시퀀스 길이보다 문장이 짧을 경우 0으로 패딩(padding)을 채워 넣은 것입니다. 사전에는 없지만 0은 바로 패딩 문자 <pad>가 될 것입니다.

### 1-3-3. 데이터셋 객체 생성

`Rock_Scissor_Paper`, `MNIST handwritten digit classification` 등  
 : model.fit(x_train, y_train, ...) 형태로 Numpy Array 데이터셋을 생성하고, model에 제공해 train  

**텐서플로우를 활용할 경우** 텐서로 생성된 데이터를 이용해 `tf.data.Dataset`객체를 생성함  
`tf.data.Dataset` 객체는 텐서플로우에서 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의 기능을 제공함

In [10]:
# tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성

BUFFER_SIZE = len(src_input) # 총 문장 수 : 24015
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>

## 1-2. 인공지능(model) 학습시키기

`tf.keras.Model`을 Subclassing하는 방식으로 model 구현  

**구현할 model** =  1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성  
.  
![image](https://user-images.githubusercontent.com/45934191/126585310-e8aa962e-4aa0-4d1e-99b9-090ac4dbc024.png)


In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256 # 워드 벡터의 차원수
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


`Embedding 레이어` : (입력 텐서의) **인덱스** 값 -> 해당 인덱스 번째의 **워드 벡터**로 변환
  - 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현 (representation)으로 사용됨

`embedding_size `  
- 워드 벡터의 차원수. 즉 단어가 추상적으로 표현되는 크기 !!
- 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만, 그만큼 충분한 데이터가 주어지지 않으면 오히려 혼란을 야기함
> ex. `embedding_size` = 2  
차갑다: [0.0, 1.0]  
뜨겁다: [1.0, 0.0]  
미지근하다: [0.5, 0.5]

`hidden_size `
- `LSTM 레이어`의 `hidden state` 의 차원수
- 모델에 얼마나 많은 일꾼을 둘 것인가?
- 일꾼들은 모두 같은 데이터를 보고 각자의 생각을 가지는데, 역시 충분한 데이터가 주어지면 올바른 결정을 내리겠지만 그렇지 않으면 배가 산으로 감 

In [12]:
# model build 전( model.compile()을 호출x, model의 입력 텐서 지정x) 에 데이터 좀 태워보기 
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 20, 7001), dtype=float32, numpy=
array([[[ 5.49086486e-04,  1.66521437e-04,  1.00034005e-04, ...,
         -3.54421034e-04,  1.98820562e-04, -8.35924657e-05],
        [ 7.75427208e-04,  5.06622659e-04, -4.77696040e-06, ...,
         -5.44135924e-04,  3.73839139e-04,  3.48993563e-05],
        [ 1.04616862e-03,  1.08062127e-03,  1.63546516e-04, ...,
         -6.86582702e-04,  4.72771382e-04, -2.25202530e-04],
        ...,
        [-5.23498573e-04,  3.32579832e-03, -3.81532795e-04, ...,
         -2.90569663e-03, -4.98008390e-04, -2.25709798e-03],
        [-6.60168065e-04,  3.53993522e-03, -9.09162685e-04, ...,
         -3.32693639e-03, -5.01539616e-04, -2.56467727e-03],
        [-7.90796010e-04,  3.67940776e-03, -1.43815309e-03, ...,
         -3.73980496e-03, -4.84170479e-04, -2.84078182e-03]],

       [[ 5.49086486e-04,  1.66521437e-04,  1.00034005e-04, ...,
         -3.54421034e-04,  1.98820562e-04, -8.35924657e-05],
        [ 9.61516227e-04,  3.30343610e-04,  1.

`shape=(batch_size, return_sequence 길이, Dense 레이어 출력 차원수)`  
- `batch_size` : 256. `dataset.take(1)`를 통해서 1개의 배치를 가져옴(256개의 문장 데이터)
- `Dense레이어 출력차원수` : 7001. 7001개의 단어 중 어느 단어의 확률이 가장 높을지 모델링
- `return_sequence 길이` 
  - `tf.keras.layers.LSTM(hidden_size, return_sequences=True)`로 호출한 LSTM 레이어에서 `return_sequences=True` 라고 지정한 부분 
  - LSTM은 자신에게 입력된 시퀀스의 길이만큼 동일한 길이의 시퀀스를 출력한다는 의미
  - `return_sequences=False` 였다면 LSTM 레이어는 1개의 벡터만 출력
  > Q. 모델은 입력 데이터의 시퀀스 길이가 얼마인지 모르고, 모델을 만들면서 알려준 적도 없음. 20은 언제 알게된 것일까?  
  A. 데이터를 입력받으면서 비로소 알게 된 것. 데이터셋의 max_len이 20으로 맞춰져 있었음




In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1792256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  7176025   
Total params: 22,607,961
Trainable params: 22,607,961
Non-trainable params: 0
_________________________________________________________________


`model.summary()`를 호출해도, `Output Shape`를 정확하게 알려주지 않음  
=> 모델은 입력 시퀀스의 길이를 모르기 때문에 Output Shape를 특정할 수 없음

하지만 모델의 파라미터 사이즈는 대략 `22million` 정도로 측정됨.  
(cf. **GPT-2**의 파라미터 사이즈  1.5billion, **GPT-3**의 파라미터 사이즈=GPT-2* 100)

### **Train 🔥🔥**

In [14]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
tf.test.is_gpu_available()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Epoch 1/30
93/93 [==============================] - 20s 186ms/step - loss: 3.4717
Epoch 2/30
93/93 [==============================] - 18s 194ms/step - loss: 2.8037
Epoch 3/30
93/93 [==============================] - 19s 205ms/step - loss: 2.7003
Epoch 4/30
93/93 [==============================] - 20s 212ms/step - loss: 2.5981
Epoch 5/30
93/93 [==============================] - 19s 205ms/step - loss: 2.5317
Epoch 6/30
93/93 [==============================] - 19s 203ms/step - loss: 2.4794
Epoch 7/30
93/93 [==============================] - 19s 203ms/step - loss: 2.4220
Epoch 8/30
93/93 [==============================] - 19s 206ms/step - loss: 2.3699
Epoch 9/30
93/93 [==============================] - 19s 207ms/step - loss: 2.3198
Epoch 10/30
93/93 [==============================] - 19s 205ms/step - loss: 2.2722
Epoch 11/30
93/93 [==============================] - 19s 203ms/step - loss: 2.2256
Epoch 12/30
93/

Loss : 모델이 오답을 만들고 있는 정도  
(Loss가 1일 때 99%를 맞추고 있다는 의미는 아님).  
오답률이 감소하고 있으니 **학습이 잘 진행되고 있다** 고 해석할 수 있다!

>학습엔 15분 정도 소요됨(GPU 환경 기준)  
혹시라도 지나치게 많은 시간이 소요된다면 `tf.test.is_gpu_available()` 를 실행해 텐서플로우가 GPU를 잘 사용하고 있는지 확인하자  
**또는 런타임 > 런타임 유형 변경 > 하드웨어 가속기에서 gpu를 가속시키자**


## 1-3. 평가하기

모델이 작문을 잘하는지 컴퓨터 알고리즘이 평가하는 것은 무리가 있다.  
**작문을 시켜보고 직접 평가하자 !**

In [15]:
# init sentense -> 작문
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # test dataset 타겟 문장, 소스 문장이 없음
    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

>while의 첫 번째 루프에서 test_tensor에 <start> 하나만 들어가고, 모델이 출력으로 7001개의 단어 중 A를 골랐다고 가정.  
while의 두 번째 루프에서 test_tensor에는 <start> A가 들어가고, 모델이 그다음 B를 골랐다고 가정  
while의 세 번째 루프에서 test_tensor에는 <start> A B가 들어가고, …


In [21]:
# 문장 생성 함수 실행
generate_text(model, tokenizer, init_sentence="<start> baby")

'<start> baby in a twofold vigour lift me up <end> '

# 2. 인공지능 작사가 만들기

[Song Lyrics](https://www.kaggle.com/paultimothymooney/poetry/data) dataset 사용

## 2-1. 데이터 전처리

### 데이터 읽어오기

In [98]:
# 데이터 읽어오기
import glob

txt_file_path = '/content/drive/MyDrive/Colab Notebooks/Data-Science/data/lyricist/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


### 전처리
지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로, (너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 ?)  
문장을 토큰화 했을 때 **토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외**하자
> 토큰의 개수 = 소스, 타켓 데이터 각각 `<end>`, `<start>`를 제외한 개수

In [99]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# Tokenize
# 정제 데이터 구축
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    # if len(sentence.split()) >= 14: continue

    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)

    # sentence = can't(1) / preprocessed_sentense = can t(2)
    if len(preprocessed_sentence.split()) >= 16: continue # start, end 제외

    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]


['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>',
 '<start> had some bad love <end>',
 '<start> ooh , ooh looking for some affirmation <end>']

In [100]:
# Vectorize (언어 데이터-->숫자)
# 12000 단어를 기억할 수 있는 단어장 만들기
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2 290  28 ...   0   0   0]
 [  2 219  13 ...   0   0   0]
 [  2  25  15 ...   0   0   0]
 ...
 [  2  21  77 ...   0   0   0]
 [  2  41  26 ...   0   0   0]
 [  2  21  77 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f3ff380d150>


### 평가 데이터셋 분리

In [101]:
from sklearn.model_selection import train_test_split

# vectorize 된 tensor를 소스와 타겟으로 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(tensor[:,:-1], tensor[:, 1:], test_size=0.2)

In [102]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


### model 만들기
모델의 `Embedding Size`와 `Hidden Size`를 조절하며 10 `Epoch` 안에 `val_loss` 값을 2.2 수준으로 줄일 수 있는 모델을 설계하자

In [103]:
embedding_size = 512 # 워드 벡터의 차원수
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
# for src_sample, tgt_sample in dataset.take(1): break

# # 한 배치만 불러온 데이터를 모델에 넣어봅니다
# model(src_sample)

In [ ]:
# model.summary()


In [104]:
optimizer = tf.keras.optimizers.Adam()
tf.test.is_gpu_available()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
3901/3901 [==============================] - 523s 130ms/step - loss: 2.9572
Epoch 2/10
3901/3901 [==============================] - 506s 130ms/step - loss: 2.4203
Epoch 3/10
3901/3901 [==============================] - 505s 129ms/step - loss: 1.9897
Epoch 4/10
3901/3901 [==============================] - 506s 130ms/step - loss: 1.6376
Epoch 5/10
3901/3901 [==============================] - 506s 130ms/step - loss: 1.3869
Epoch 6/10
3901/3901 [==============================] - 503s 129ms/step - loss: 1.2260
Epoch 7/10
3901/3901 [==============================] - 507s 130ms/step - loss: 1.1338
Epoch 8/10
3901/3901 [==============================] - 509s 130ms/step - loss: 1.0831
Epoch 9/10
3901/3901 [==============================] - 507s 130ms/step - loss: 1.0535
Epoch 10/10
3901/3901 [==============================] - 507s 130ms/step - loss: 1.0341


### 평가하기

In [105]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

In [106]:
generate_text(model, tokenizer, init_sentence="<start> i", max_len=20)

'<start> i m the only one <end> '

todo

텍스트 분류 모델을 다루어 보셨다면 Embedding 레이어의 역할에 대해서는 낯설지 않을 것입니다.  